In [10]:
import dataclasses
from typing import Optional, List

import numpy as np
import plotly.subplots
import torch
from torch import nn

from swarm_nfomp.utils.math import wrap_angles
from swarm_nfomp.utils.position2d import Position2D
from swarm_nfomp.utils.position_array2d import PositionArray2D
from swarm_nfomp.utils.timer import Timer
from swarm_nfomp.utils.universal_factory import UniversalFactory

# Collision function

In [46]:
class PointArray2D:
    def __init__(self, x, y):
        assert x.shape == y.shape
        self._x = x
        self._y = y

    def as_numpy(self):
        return np.stack([self._x, self._y], axis=-1)

    @classmethod
    def from_vec(cls, vec):
        return cls(vec[..., 0], vec[..., 1])

    @property
    def x(self):
        return self._x

    @property
    def y(self):
        return self._y

    def __repr__(self):
        return f"PointArray2D(x={self.x}, y={self.y})"


class RectangleRegionArray:
    def __init__(self, min_x: np.ndarray, max_x: np.ndarray, min_y: np.ndarray, max_y: np.ndarray):
        assert min_x.shape == max_x.shape
        assert min_x.shape == min_y.shape
        assert min_x.shape == max_y.shape
        assert np.all(min_x < max_x)
        assert np.all(min_y < max_y)
        self.min_x = min_x
        self.max_x = max_x
        self.min_y = min_y
        self.max_y = max_y

    def inside(self, positions: PointArray2D) -> np.ndarray:
        result = self.min_x <= positions.x[:, None]
        result &= positions.x[:, None] <= self.max_x
        result &= self.min_y <= positions.y[:, None]
        result &= positions.y[:, None] <= self.max_y
        return np.any(result, axis=1)

    @classmethod
    def from_dict(cls, data):
        data = np.array(data)
        assert len(data.shape) == 2
        assert data.shape[1] == 4
        return cls(data[:, 0], data[:, 1], data[:, 2], data[:, 3])

    def __len__(self):
        return len(self.min_x)


class RectangleRegionCollisionDetector:
    def __init__(self, inside_rectangle_region_array: RectangleRegionArray,
                 outside_rectangle_region_array: RectangleRegionArray):
        self.inside_rectangle_region_array = inside_rectangle_region_array
        self.outside_rectangle_region_array = outside_rectangle_region_array

    def is_collision(self, positions: PointArray2D) -> np.ndarray:
        return self.inside_rectangle_region_array.inside(positions) | (
            ~self.outside_rectangle_region_array.inside(positions))

    @classmethod
    def from_dict(cls, data):
        return RectangleRegionCollisionDetector(RectangleRegionArray.from_dict(data["inside_rectangle_region_array"]),
                                                RectangleRegionArray.from_dict(data["outside_rectangle_region_array"]))


In [47]:
import itertools


def flat_lists(list_to_flat):
    return list(itertools.chain.from_iterable(list_to_flat))


class RectangleRegionCollisionDetectorVisualizer:
    def visualize(self, detector: RectangleRegionCollisionDetector):
        fig = plotly.subplots.make_subplots(rows=1, cols=1)
        self.get_region_scatter(fig, detector.inside_rectangle_region_array, fill="toself")
        self.get_region_scatter(fig, detector.outside_rectangle_region_array)
        fig.update_yaxes(scaleanchor="x", scaleratio=1)
        return fig

    @staticmethod
    def get_region_scatter(fig, region, **kwargs):
        x_lists = [[region.min_x[i], region.max_x[i], region.max_x[i], region.min_x[i], region.min_x[i], None] for i in
                   range(len(region))]
        y_lists = [[region.min_y[i], region.min_y[i], region.max_y[i], region.max_y[i], region.min_y[i], None] for i in
                   range(len(region))]
        x = flat_lists(x_lists)
        y = flat_lists(y_lists)
        fig.add_scatter(x=x[:-1], y=y[:-1], mode="lines", **kwargs)


In [53]:
points = np.random.rand(1000, 2)
points[:, 0] = 5 + (points[:, 0] - 0.5) * 2 * 18
points[:, 1] = 5 + (points[:, 1] - 0.5) * 2 * 6
points = PointArray2D.from_vec(points)
parameters = {
    "outside_rectangle_region_array": [[-10, 20, 0, 10]], "inside_rectangle_region_array": [[4, 6, 0, 4], [4, 6, 6, 10]]
}
collision_detector = RectangleRegionCollisionDetector.from_dict(parameters)
mask = collision_detector.is_collision(points)
fig = RectangleRegionCollisionDetectorVisualizer().visualize(collision_detector)
fig.add_scatter(x=points.as_numpy()[:, 0][mask], y=points.as_numpy()[:, 1][mask], name="collision", mode="markers")
fig.add_scatter(x=points.as_numpy()[:, 0][~mask], y=points.as_numpy()[:, 1][~mask], name="non_collision",
                mode="markers")
fig.show()

In [54]:
@dataclasses.dataclass
class ONFModelConfig:
    mean: float
    sigma: float
    input_dimension: int
    encoding_dimension: int
    hidden_dimensions: List[int]


class ONF(nn.Module):
    def __init__(self, parameters: ONFModelConfig):
        super().__init__()
        self.encoding_layer = nn.Linear(parameters.input_dimension, parameters.encoding_dimension)
        self.mlp1 = self.make_mlp(parameters.encoding_dimension, parameters.hidden_dimensions[:-1],
                                  parameters.hidden_dimensions[-1])
        self.mlp2 = self.make_mlp(parameters.encoding_dimension + parameters.hidden_dimensions[-1], [], 1)
        self._mean = parameters.mean
        self._sigma = parameters.sigma

    @classmethod
    def make_mlp(cls, input_dimension, hidden_dimensions, output_dimension):
        modules = []
        for dimension in hidden_dimensions:
            modules.append(nn.Linear(input_dimension, dimension))
            modules.append(nn.ReLU())
            input_dimension = dimension
        modules.append(nn.Linear(input_dimension, output_dimension))
        return nn.Sequential(*modules)

    def forward(self, x):
        x = (x - self._mean) / self._sigma
        x = self.encoding_layer(x)
        x = torch.sin(x)
        input_x = x
        x = self.mlp1(input_x)
        x = torch.cat([x, input_x], dim=1)
        x = self.mlp2(x)
        return x


In [59]:
@dataclasses.dataclass
class SwarmState:
    middle_position: Position2D
    drone_configuration_points: PointArray2D


@dataclasses.dataclass
class SwarmPathPlannerTask:
    start: SwarmState
    goal: SwarmState
    collision_detector: RectangleRegionCollisionDetector


@dataclasses.dataclass
class ResultPath:
    middle_positions: PositionArray2D
    drone_configuration_points: PointArray2D

    @property
    def drone_global_positions(self) -> np.ndarray:
        drone_configuration_points = self.drone_configuration_points.as_numpy()
        global_points = self.middle_positions.apply(drone_configuration_points.transpose((1, 0, 2)))
        global_points = global_points.transpose((1, 0, 2))
        return global_points


@dataclasses.dataclass
class PathOptimizedState:
    middle_positions: torch.Tensor
    drone_configuration_points: torch.Tensor
    start_drone_configuration: torch.Tensor
    goal_drone_configuration: torch.Tensor

    @property
    def result_path(self) -> ResultPath:
        return ResultPath(
            middle_positions=PositionArray2D.from_vec(self.middle_positions.cpu().detach().numpy()),
            drone_configuration_points=PointArray2D.from_vec(self.drone_configuration_points.cpu().detach().numpy())
        )

    @property
    def full_drone_configuration(self) -> torch.Tensor:
        return torch.cat([self.start_drone_configuration, self.drone_configuration_points, self.goal_drone_configuration], dim=0)


@dataclasses.dataclass
class OptimizerImplConfig:
    lr: float
    beta1: float
    beta2: float


class OptimizerImpl:
    def __init__(self, parameters: OptimizerImplConfig):
        self._optimizer = None
        self._parameters = parameters

    def setup(self, model_parameters):
        self._optimizer = torch.optim.Adam(model_parameters, lr=self._parameters.lr,
                                           betas=(self._parameters.beta1, self._parameters.beta2))

    def zero_grad(self):
        self._optimizer.zero_grad()

    def step(self):
        self._optimizer.step()


class PathOptimizedStateInitializer:
    def __init__(self, planner_task: SwarmPathPlannerTask, path_state_count: int, device: str):
        self._planner_task = planner_task
        self._path_state_count = path_state_count
        self._device = device

    def init(self) -> PathOptimizedState:
        start_configuration = torch.tensor(self._planner_task.start.drone_configuration_points.as_numpy())
        drone_configuration_points = torch.zeros(self._path_state_count, *start_configuration.shape, requires_grad=True,
                                                 device=self._device)
        with torch.no_grad():
            drone_configuration_points.data = torch.repeat_interleave(start_configuration[None], self._path_state_count,
                                                                      dim=0)
        return PathOptimizedState(
            middle_positions=self._initialize_trajectory(),
            drone_configuration_points=drone_configuration_points,
            start_drone_configuration=torch.tensor(self._planner_task.start.drone_configuration_points, requires_grad=False)[None],
            goal_drone_configuration=torch.tensor(self._planner_task.goal.drone_configuration_points, requires_grad=False)[None]
        )

    def _initialize_trajectory(self) -> torch.Tensor:
        trajectory_length = self._path_state_count + 2
        trajectory = torch.zeros(self._path_state_count, 3, requires_grad=False, device=self._device)
        start_point: np.ndarray = self._planner_task.start.middle_position.translation
        goal_point: np.ndarray = self._planner_task.goal.middle_position.translation
        trajectory[:, 0] = torch.linspace(start_point[0], goal_point[0], trajectory_length)[1:-1]
        trajectory[:, 1] = torch.linspace(start_point[1], goal_point[1], trajectory_length)[1:-1]
        trajectory[:, 2] = self._initialize_angles()
        return trajectory

    def _initialize_angles(self) -> torch.Tensor:
        start_angle: float = self._planner_task.start.middle_position.rotation
        goal_angle: float = self._planner_task.goal.middle_position.rotation
        trajectory_length = self._path_state_count + 2
        delta_angle = wrap_angles(goal_angle - start_angle)
        goal_angle = delta_angle + start_angle
        return torch.linspace(start_angle, goal_angle, trajectory_length)[1:-1]

@dataclasses.dataclass
class PathLossBuilderConfig:
    distance_weight: float
    collision_weight: float
    configuration_weight: float

class PathLossBuilder:
    def __init__(self, parameters: PathLossBuilderConfig):
        self._parameters = parameters

    def get_loss(self, collision_model: nn.Module, optimized_state: PathOptimizedState):
        loss = self._parameters.distance_weight * self._distance_loss(optimized_state)
        loss = loss + self._parameters.collision_weight * self.collision_loss(collision_model, optimized_state)
        loss = loss + self._parameters.configuration_weight * self._configuration_loss(optimized_state)
        return loss

    @staticmethod
    def _distance_loss(optimized_state):
        path = optimized_state.full_drone_configuration
        return torch.mean((path[1:] - path[:-1]) ** 2)

class GradPreconditioner:
    @staticmethod
    def precondition(optimized_state: PathOptimizedState):
        return optimized_state


class PathOptimizer:
    def __init__(self, timer: Timer, optimizer: OptimizerImpl, loss_builder: PathLossBuilder,
                 state_initializer: PathOptimizedStateInitializer, grad_preconditioner: GradPreconditioner):
        self._loss_builder = loss_builder
        self._optimizer = optimizer
        self._timer = timer
        self._grad_preconditioner = grad_preconditioner
        self._state_initializer = state_initializer
        self._optimized_state: Optional[PathOptimizedState] = None

    def setup(self):
        self._optimized_state = self._state_initializer.init()
        self._optimizer.setup([self._optimized_state.drone_configuration_points])

    def step(self, collision_model):
        self._optimizer.zero_grad()
        loss = self._loss_builder.get_loss(collision_model, self._optimized_state)
        self._timer.tick("trajectory_backward")
        loss.backward()
        self._timer.tock("trajectory_backward")
        self._timer.tick("inv_hes_grad_multiplication")
        self._grad_preconditioner.precondition(self._optimized_state)
        self._timer.tock("inv_hes_grad_multiplication")
        self._timer.tick("trajectory_optimizer_step")
        self._optimizer.step()
        self._timer.tock("trajectory_optimizer_step")

    @property
    def result_path(self) -> ResultPath:
        return self._optimized_state.result_path


class CollisionModelPointSampler:
    def __init__(self, fine_random_offset: float, course_random_offset: float):
        self._fine_random_offset = fine_random_offset
        self._course_random_offset = course_random_offset

    def sample(self, path: ResultPath) -> PointArray2D:
        points: np.ndarray = path.drone_global_positions.reshape(-1, 2)
        course_points = points + np.random.randn(*points.shape) * self._course_random_offset
        fine_points = points + np.random.randn(*points.shape) * self._fine_random_offset
        points = np.concatenate([course_points, fine_points], axis=0)
        print(points.shape)
        return PointArray2D.from_vec(points)


class CollisionModelFactory:
    def __init__(self, parameters: ONFModelConfig):
        self._parameters = parameters

    def make_collision_model(self):
        return ONF(self._parameters)


class CollisionNeuralFieldModelTrainer:
    def __init__(self, timer: Timer, planner_task: SwarmPathPlannerTask,
                 collision_model_point_sampler: CollisionModelPointSampler,
                 optimizer: OptimizerImpl, collision_model_factory: CollisionModelFactory,
                 device: str):
        self._timer = timer
        self._collision_model_point_sampler = collision_model_point_sampler
        self._optimizer = optimizer
        self._collision_model_factory = collision_model_factory
        self._planner_task = planner_task
        self._collision_model: Optional[nn.Module] = None
        self._device = device
        self._collision_loss_function = nn.BCEWithLogitsLoss()

    def setup(self):
        self._collision_model = self._collision_model_factory.make_collision_model()
        self._optimizer.setup(self._collision_model.parameters())

    def learning_step(self, path: ResultPath):
        timer.tick("optimize_collision_model")
        points = self._collision_model_point_sampler.sample(path)
        self._collision_model.requires_grad_(True)
        self._optimizer.zero_grad()
        predicted_collision = self._calculate_predicted_collision(points)
        truth_collision = self._calculate_truth_collision(points)
        truth_collision = torch.tensor(truth_collision.astype(np.float32)[:, None], device=self._device)
        loss = self._collision_loss_function(predicted_collision, truth_collision)
        loss.backward()
        self._optimizer.step()
        self._collision_model.requires_grad_(False)
        timer.tock("optimize_collision_model")

    def _calculate_predicted_collision(self, points: PointArray2D):
        return self._collision_model(torch.tensor(points.as_numpy().astype(np.float32), device=self._device))

    def _calculate_truth_collision(self, points):
        return self._planner_task.collision_detector.is_collision(points)

    @property
    def collision_model(self) -> nn.Module:
        return self._collision_model


class SwarmNeuralFieldOptimalPathPlanner:
    def __init__(self, planner_task: SwarmPathPlannerTask,
                 collision_neural_field_model_trainer: CollisionNeuralFieldModelTrainer, path_optimizer: PathOptimizer,
                 iterations: int):
        self._parameters = parameters
        self._planner_task = planner_task
        self._path_optimizer = path_optimizer
        self._collision_neural_field_model_trainer = collision_neural_field_model_trainer
        self._iterations = iterations

    def plan(self) -> ResultPath:
        self._path_optimizer.setup()
        self._collision_neural_field_model_trainer.setup()
        for i in range(self._iterations):
            self.step()
        return self._path_optimizer.result_path

    def step(self):
        path: ResultPath = self._path_optimizer.result_path
        self._collision_neural_field_model_trainer.learning_step(path)
        collision_model = self._collision_neural_field_model_trainer.collision_model
        self._path_optimizer.step(collision_model)

In [60]:
collision_detector_parameters = {
    "outside_rectangle_region_array": [[-10, 20, 0, 10]], "inside_rectangle_region_array": [[4, 6, 0, 4], [4, 6, 6, 10]]
}
collision_detector = RectangleRegionCollisionDetector.from_dict(collision_detector_parameters)
drone_configuration = PointArray2D(np.array([0., -2, 2, 0, 0]), np.array([0, 0, 0, -2, 2]))
planner_task = SwarmPathPlannerTask(
    start=SwarmState(Position2D(-5, 5, 0), drone_configuration),
    goal=SwarmState(Position2D(15, 5, 0), drone_configuration),
    collision_detector=collision_detector
)

planner_parameters = {
    "collision_neural_field_model_trainer": {
        "collision_model_point_sampler": {
            "course_random_offset": 5.,
            "fine_random_offset": 1.,
        },
        "optimizer": {
            "lr": 5e-2,
            "beta1": 0.9,
            "beta2": 0.9
        },
        "collision_model_factory": {
            "mean": 0,
            "sigma": 1.,
            "input_dimension": 2,
            "encoding_dimension": 64,
            "hidden_dimensions": [64, 64]
        }
    },
    "path_optimizer": {
        "optimizer": {
            "lr": 1e-2,
            "beta1": 0.9,
            "beta2": 0.9
        },
        "loss_builder": {},
        "state_initializer": {
            "path_state_count": 100,
        },
        "grad_preconditioner": {}
    },
    "iterations": 1,
}
timer = Timer()
device = "cpu"
swarm_path_planner = UniversalFactory().make(SwarmNeuralFieldOptimalPathPlanner, planner_parameters,
                                             planner_task=planner_task, timer=timer, device=device)
swarm_path = swarm_path_planner.plan()

(1000, 2)


AttributeError: 'PathLossBuilder' object has no attribute 'get_loss'

In [ ]:
swarm_path.drone_configuration_points.as_numpy().shape